In [ ]:
# default_exp models.conv_rnn

# Recurrent Convolutional Kernels
> ConvLSTM and ConvGRU cells and models

In [ ]:
#export
from fastai2.vision.all import *

In [ ]:
if torch.cuda.is_available():
    torch.cuda.set_device(1)
    print(torch.cuda.get_device_name())

GeForce RTX 2070 SUPER


In [ ]:
#export
class ConvGRU_cell(Module):
    def __init__(self, in_ch, out_ch, ks=3, debug=False):
        self.in_ch = in_ch
        # kernel_size of input_to_state equals state_to_state
        self.ks = ks
        self.out_ch = out_ch
        self.debug = debug
        self.padding = (ks - 1) // 2
        self.conv1 = nn.Sequential(nn.Conv2d(self.in_ch + self.out_ch,2 * self.out_ch, self.ks, 1,self.padding),
                                   nn.GroupNorm(2 * self.out_ch // 8, 2 * self.out_ch))
        self.conv2 = nn.Sequential(nn.Conv2d(self.in_ch + self.out_ch,self.out_ch, self.ks, 1, self.padding),
                                   nn.GroupNorm(self.out_ch // 8, self.out_ch))

    def forward(self, inputs, hidden_state=None):
        "inputs shape: (bs, seq_len, ch, w, h)"
        bs, seq_len, ch, w, h = inputs.shape
        if hidden_state is None:
            htprev = self.initHidden(bs, self.out_ch, w, h)
            if self.debug: print(f'htprev: {htprev.shape}')
        else:
            htprev = hidden_state
        output_inner = []
        for index in range(seq_len):
            x = inputs[:, index, ...]
            combined_1 = torch.cat((x, htprev), 1)  # X_t + H_t-1
            gates = self.conv1(combined_1)  # W * (X_t + H_t-1)          
            zgate, rgate = torch.split(gates, self.out_ch, dim=1)
            z = torch.sigmoid(zgate)
            r = torch.sigmoid(rgate)
            combined_2 = torch.cat((x, r * htprev),1)
            ht = self.conv2(combined_2)
            ht = torch.tanh(ht)
            htnext = (1 - z) * htprev + z * ht
            output_inner.append(htnext)
            htprev = htnext
        return torch.stack(output_inner, dim=1), htnext
    def __repr__(self): return f'ConvGRU_cell(in={self.in_ch}, out={self.out_ch}, ks={self.ks})'
    def initHidden(self, bs, ch, w, h): return one_param(self).new_zeros(bs, ch, w, h)

Let's check:

In [ ]:
cell = ConvGRU_cell(32, 32, debug=True)
cell

ConvGRU_cell(in=32, out=32, ks=3)

In [ ]:
x = torch.rand(2, 7, 32, 64, 64)
out, h = cell(x)

htprev: torch.Size([2, 32, 64, 64])


In [ ]:
out.shape

torch.Size([2, 7, 32, 64, 64])

Checking sizes:

In [ ]:
test_eq(out.shape, x.shape) 
test_eq(h.shape, [2,32,64,64])

Should be possible to call with hidden state:

In [ ]:
out2, h2 = cell(out, h)
test_eq(h2.shape, [2, 32, 64, 64])

A very nasty module to propagate 2D layers over sequence of images, inspired from Keras

In [ ]:
#export
class TimeDistributed(Module):
    "Applies a module over tdim identically for each step" 
    def __init__(self, module, low_mem=False, tdim=1):
        self.module = module
        self.low_mem = low_mem
        self.tdim = tdim
        
    def forward(self, *args, **kwargs):
        "input x with shape:(bs,seq_len,channels,width,height)"
        if self.low_mem or self.tdim!=1: 
            return self.low_mem_forward(*args)
        else:
            #only support tdim=1
            inp_shape = args[0].shape
            bs, seq_len = inp_shape[0], inp_shape[1]   
            out = self.module(*[x.view(bs*seq_len, *x.shape[2:]) for x in args], **kwargs)
            out_shape = out.shape
            return out.view(bs, seq_len,*out_shape[1:])
    
    def low_mem_forward(self, *args, **kwargs):                                           
        "input x with shape:(bs,seq_len,channels,width,height)"
        tlen = args[0].shape[self.tdim]
        args_split = [torch.unbind(x, dim=self.tdim) for x in args]
        out = []
        for i in range(tlen):
            out.append(self.module(*[args[i] for args in args_split]), **kwargs)
        return torch.stack(out,dim=self.tdim)
    def __repr__(self):
        return f'TimeDistributed({self.module})'

In [ ]:
class Dummy(Module):
    def __init__(self): pass
    def forward(self, x, y): return x+y

In [ ]:
tdconv = TimeDistributed(nn.Conv2d(2, 5, 3, 1, 1))
tdconv

TimeDistributed(Conv2d(2, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))

In [ ]:
tdconv.low_mem_forward(torch.rand(3, 10, 2, 8, 8)).shape

torch.Size([3, 10, 5, 8, 8])

In [ ]:
tconv2 = TimeDistributed(Dummy())

In [ ]:
tconv2.low_mem_forward(torch.rand(3, 10, 5), torch.rand(3, 10, 5)).shape

torch.Size([3, 10, 5])

In [ ]:
tdconv(torch.rand(3, 10, 2, 8, 8)).shape

torch.Size([3, 10, 5, 8, 8])

## Encoder

In [ ]:
#export
class Encoder(Module):
    def __init__(self, n_in=1, szs=[16,64,96], ks=3, rnn_ks=5, act=nn.ReLU, norm=None, debug=False):
        self.debug = debug
        convs = []
        rnns = []
        szs = [n_in]+szs
        for ni, nf in zip(szs[0:-1], szs[1:]):
            convs.append(ConvLayer(ni, nf, ks=ks, stride=1 if ni==n_in else 2, padding=ks//2, act_cls=act, norm_type=norm))
            rnns.append(ConvGRU_cell(nf, nf, ks=rnn_ks))
        self.convs = nn.ModuleList(TimeDistributed(conv) for conv in convs)
        self.rnns = nn.ModuleList(rnns)
        
    def forward_by_stage(self, inputs, conv, rnn):
        if self.debug: 
            print(f' Layer: {rnn}')
            print(' inputs: ', inputs.shape)
        inputs = conv(inputs)
        if self.debug: print(' after_convs: ', inputs.shape)
        outputs_stage, state_stage = rnn(inputs, None)
        if self.debug: print(' output_stage: ', outputs_stage.shape)
        return outputs_stage, state_stage

    def forward(self, inputs):
        "inputs.shape bs,seq_len,1,64,64"
        hidden_states = []
        outputs = []
        for i, (conv, rnn) in enumerate(zip(self.convs, self.rnns)):
            if self.debug: print('stage: ',i)
            inputs, state_stage = self.forward_by_stage(inputs, conv, rnn)
            outputs.append(inputs)
            hidden_states.append(state_stage)
        return outputs, hidden_states

In [ ]:
enc = Encoder(debug=True)
enc

Encoder(
  (convs): ModuleList(
    (0): TimeDistributed(ConvLayer(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    ))
    (1): TimeDistributed(ConvLayer(
      (0): Conv2d(16, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
    ))
    (2): TimeDistributed(ConvLayer(
      (0): Conv2d(64, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
    ))
  )
  (rnns): ModuleList(
    (0): ConvGRU_cell(in=16, out=16, ks=5)
    (1): ConvGRU_cell(in=64, out=64, ks=5)
    (2): ConvGRU_cell(in=96, out=96, ks=5)
  )
)

In [ ]:
imgs = torch.rand(2, 10, 1, 64, 64)

In [ ]:
enc_out, h = enc(imgs)

stage:  0
 Layer: ConvGRU_cell(in=16, out=16, ks=5)
 inputs:  torch.Size([2, 10, 1, 64, 64])
 after_convs:  torch.Size([2, 10, 16, 64, 64])
 output_stage:  torch.Size([2, 10, 16, 64, 64])
stage:  1
 Layer: ConvGRU_cell(in=64, out=64, ks=5)
 inputs:  torch.Size([2, 10, 16, 64, 64])
 after_convs:  torch.Size([2, 10, 64, 32, 32])
 output_stage:  torch.Size([2, 10, 64, 32, 32])
stage:  2
 Layer: ConvGRU_cell(in=96, out=96, ks=5)
 inputs:  torch.Size([2, 10, 64, 32, 32])
 after_convs:  torch.Size([2, 10, 96, 16, 16])
 output_stage:  torch.Size([2, 10, 96, 16, 16])


In [ ]:
[_.shape for _ in h]

[torch.Size([2, 16, 64, 64]),
 torch.Size([2, 64, 32, 32]),
 torch.Size([2, 96, 16, 16])]

In [ ]:
[_.shape for _ in enc_out]

[torch.Size([2, 10, 16, 64, 64]),
 torch.Size([2, 10, 64, 32, 32]),
 torch.Size([2, 10, 96, 16, 16])]

## Decoder

In [ ]:
#export 
class UpsampleBlock(Module):
    "A quasi-UNet block, using `PixelShuffle_ICNR upsampling`."
    @delegates(ConvLayer.__init__)
    def __init__(self, in_ch, out_ch, residual=True, blur=False, act_cls=defaults.activation,
                 self_attention=False, init=nn.init.kaiming_normal_, norm_type=None, debug=False, **kwargs):
        store_attr(self, 'in_ch,out_ch,residual, blur,act_cls,self_attention,norm_type,debug')
        self.shuf = PixelShuffle_ICNR(in_ch, in_ch//2, blur=blur, act_cls=act_cls, norm_type=norm_type)
        ni = in_ch//2 if not residual else in_ch//2 + out_ch  #the residual has out_ch (normally in_ch//2)
        nf = out_ch
        self.conv1 = ConvLayer(ni, nf, act_cls=act_cls, norm_type=norm_type, **kwargs)
        self.conv2 = ConvLayer(nf, nf, act_cls=act_cls, norm_type=norm_type,
                               xtra=SelfAttention(nf) if self_attention else None, **kwargs)
        self.bn = nn.BatchNorm2d(out_ch)
        self.relu = act_cls()
        apply_init(nn.Sequential(self.conv1, self.conv2), init)
    def __repr__(self): return (f'UpsampleBLock(in={self.in_ch}, out={self.out_ch}, blur={self.blur}, residual={self.residual}, '
                                f'act={self.act_cls()}, attn={self.self_attention}, norm={self.norm_type})')
    def forward(self, up_in, side_in=None):
        up_out = self.shuf(up_in)
        if side_in is not None:
            if self.debug: print(f'up_out: {up_out.shape}, side_in: {side_in.shape}')
            assert up_out.shape[-2:] == side_in.shape[-2::], 'residual shape does not match input'
            up_out = torch.cat([up_out, self.bn(side_in)], dim=1)
        if self.debug: print(f'up_out: {up_out.shape}')
        return self.conv2(self.conv1(up_out))

In [ ]:
us = UpsampleBlock(32, 16, residual=False)
us

UpsampleBLock(in=32, out=16, blur=False, residual=False, act=ReLU(), attn=False, norm=None)

In [ ]:
us(torch.rand(8, 32, 32, 32)).shape

torch.Size([8, 16, 64, 64])

In [ ]:
us_residual = UpsampleBlock(32, 16, residual=True, debug=True)
us

UpsampleBLock(in=32, out=16, blur=False, residual=False, act=ReLU(), attn=False, norm=None)

In [ ]:
us_residual(torch.rand(8, 32, 32, 32), side_in=torch.rand(8, 16, 64, 64)).shape

up_out: torch.Size([8, 16, 64, 64]), side_in: torch.Size([8, 16, 64, 64])
up_out: torch.Size([8, 32, 64, 64])


torch.Size([8, 16, 64, 64])

it can take a residual connection

In [ ]:
#export
class Decoder(Module):
    def __init__(self, n_out=1, szs=[64,32,16], ks=3, rnn_ks=5, act=nn.ReLU, blur=False, attn=False, 
                 norm=None, debug=False):
        self.debug = debug
        deconvs = []
        rnns = []
        szs = szs
        for ni, nf in zip(szs[0:-1], szs[1:]):
            deconvs.append(UpsampleBlock(ni, nf, blur=blur, self_attention=attn, act_cls=act, norm_type=norm))
            rnns.append(ConvGRU_cell(ni, ni, ks=rnn_ks))
        
        #last layer
        deconvs.append(ConvLayer(szs[-1], szs[-1], ks, padding=ks//2, act_cls=act, norm_type=norm))
        self.deconvs = nn.ModuleList(TimeDistributed(conv) for conv in deconvs)
        self.rnns = nn.ModuleList(rnns)
        self.head = TimeDistributed(nn.Conv2d(szs[-1], n_out, kernel_size=1))

    def forward_by_stage(self, inputs, state, deconv, rnn, side_in=None):
        if self.debug: 
            print(f' Layer: {rnn}')
            print(' inputs:, state: ', inputs.shape, state.shape)
        inputs, state_stage = rnn(inputs, state)
        if self.debug: 
            print(' after rnn: ', inputs.shape)
            print(f' Layer: {deconv}')
        print(f' before Upsample: {inputs.shape, side_in.shape}')
        outputs_stage = deconv(inputs, side_in)
        if self.debug: print(' after_deconvs: ', outputs_stage.shape)
        return outputs_stage, state_stage
    
    def forward(self, dec_input, hidden_states, enc_outs):
        for i, (state, conv, rnn, enc_out) in enumerate(zip(hidden_states[::-1], self.deconvs, self.rnns, enc_outs[::-1])):
            if self.debug: print(f'\nStage: {i} ---------------------------------')
            dec_input, state_stage = self.forward_by_stage(dec_input, state, conv, rnn, side_in=enc_out)
        return self.head(dec_input)

In [ ]:
dec = Decoder(debug=True)
dec

Decoder(
  (deconvs): ModuleList(
    (0): TimeDistributed(UpsampleBLock(in=64, out=32, blur=False, residual=True, act=ReLU(), attn=False, norm=None))
    (1): TimeDistributed(UpsampleBLock(in=32, out=16, blur=False, residual=True, act=ReLU(), attn=False, norm=None))
    (2): TimeDistributed(ConvLayer(
      (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    ))
  )
  (rnns): ModuleList(
    (0): ConvGRU_cell(in=64, out=64, ks=5)
    (1): ConvGRU_cell(in=32, out=32, ks=5)
  )
  (head): TimeDistributed(Conv2d(16, 1, kernel_size=(1, 1), stride=(1, 1)))
)

In [ ]:
dec(enc_out[-1], h, enc_out)


Stage: 0 ---------------------------------
 Layer: ConvGRU_cell(in=64, out=64, ks=5)
 inputs:, state:  torch.Size([2, 10, 96, 16, 16]) torch.Size([2, 96, 16, 16])


RuntimeError: Given groups=1, weight of size [128, 128, 5, 5], expected input[2, 192, 16, 16] to have 128 channels, but got 192 channels instead

In [ ]:
test_eq(dec(enc_out, h).shape, imgs.shape)
print('\n--with image list--\n')
test_eq(dec(*enc(imgs)).shape, imgs.shape)

TypeError: forward() missing 1 required positional argument: 'enc_outs'

## Model

In [ ]:
#export
class StackUnstack(Module):
    "Stack together inputs, apply module, unstack output"
    def __init__(self, module, dim=1):
        self.dim = dim
        self.module = module
    def forward(self, x):
        if isinstance(x, list) or isinstance(x, tuple):
            x = torch.stack(x, dim=self.dim)
            x = self.module(x)
            return x.unbind(dim=self.dim)
        else: return self.module(x)

In [ ]:
#export
class SimpleModel(Module):
    "Simple Encoder/Decoder module"
    def __init__(self, n_in=1, n_out=1, szs=[16,64,96,96], ks=3, rnn_ks=5, act=nn.ReLU, blur=False, attn=False, 
                 norm=None, strategy='zero', debug=False):
        self.strategy = strategy
        self.encoder = Encoder(n_in, szs, ks, rnn_ks, act, norm, debug)
        self.decoder = Decoder(n_out, szs, ks, rnn_ks, act, blur, attn, norm, debug)
    def forward(self, x):
        enc_out, h = self.encoder(x)
        if self.strategy is 'zero':
            dec_in = one_param(self).new_zeros(*enc_out.shape)
        elif self.strategy is 'encoder':
            dec_in = enc_out
        return self.decoder(dec_in, h)

In [ ]:
m = StackUnstack(SimpleModel(strategy='zero'))

In [ ]:
imgs_list = [torch.rand(2,1,64,64) for _ in range(10)]

In [ ]:
test_eq(len(m(imgs_list)), len(imgs_list))

In [ ]:
test_eq(m(imgs).shape, imgs.shape)

In [ ]:
out = m(imgs)

In [ ]:
out.shape

## Loss

As the model can output a list of tensors, we will need to modify the loss function to acomodate this inputs.

In [ ]:
#export
def StackLoss(loss_func=MSELossFlat(), axis=1):
    def _inner_loss(x,y):
        x = torch.stack(x, axis)
        y = torch.stack(y, axis)
        print(x.shape, y.shape)
        return loss_func(x,y)
    return _inner_loss

In [ ]:
loss_func = StackLoss()

In [ ]:
loss_func(imgs_list, m(imgs_list))

In [ ]:
ce_stack = StackLoss(CrossEntropyLossFlat(axis=2))

In [ ]:
ce_stack([torch.rand(32, 2, 64, 64) for _ in range(10)], [torch.randint(0,1, (32,1,64,64)) for _ in range(10)])

# Export -

In [ ]:
# hide
from nbdev.export import *
notebook2script()